In [112]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import random

In [113]:
class FeatureDataset(torch.utils.data.Dataset):
    def __init__(self, image_features, text_features):
        self.image_features = image_features
        self.text_features = text_features

    def __len__(self):
        return len(self.image_features)

    def __getitem__(self, index):
        return self.image_features[index], self.text_features[index]

In [114]:
class BranchNetwork(nn.Module):
    def __init__(self, img_feature_size, txt_feature_size, fc_dim = 2048, embed_dim = 512):
        super(BranchNetwork, self).__init__()
        self.img_fc = self._branch_network(img_feature_size, fc_dim, embed_dim)
        self.txt_fc = self._branch_network(txt_feature_size, fc_dim, embed_dim)
    
    def _branch_network(self, input_dim, fc_dim, embed_dim):
        return nn.Sequential(
            nn.Linear(input_dim, fc_dim),
            nn.BatchNorm1d(fc_dim, momentum=0.1, eps=1e-5),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(fc_dim, embed_dim)
        )

    def forward(self, image_features, text_features):
        img_out = self.img_fc(image_features)
        img_embedding = F.normalize(img_out, p=2, dim=1, eps=1e-10)
        txt_out = self.txt_fc(text_features)
        txt_embedding = F.normalize(txt_out, p=2, dim=1, eps=1e-10)

        return img_embedding, txt_embedding

In [160]:
class EmbeddingLoss(nn.Module):
    def __init__(self, margin=0.2, num_neg_sample=5, im_loss_factor=0.5, txt_only_loss_factor=0.01):
        super(EmbeddingLoss, self).__init__()

        self.im_loss_factor = im_loss_factor
        self.txt_only_loss_factor = txt_only_loss_factor
        self.margin = margin
        self.num_neg_sample = num_neg_sample

    def forward(self, im_embeds, txt_embeds, im_labels, sample_size):
        txt_im_ratio = sample_size
        num_img = im_embeds.shape[0]
        num_txt = num_img * txt_im_ratio

        txt_im_dist = self._pdist(txt_embeds, im_embeds)

        # Image loss
        pos_pair_dist = txt_im_dist[im_labels].view(num_img, 1)
        neg_pair_dist = txt_im_dist[~im_labels].view(num_img, -1)
        im_loss = F.relu(self.margin + pos_pair_dist - neg_pair_dist).topk(self.num_neg_sample)[0].mean()

        # Sentence loss
        neg_pair_dist = txt_im_dist.t()[~im_labels.t()].view(num_img, -1).repeat(1, txt_im_ratio)
        txt_loss = F.relu(self.margin + pos_pair_dist - neg_pair_dist).topk(self.num_neg_sample)[0].mean()

        # Sentence only loss
        txt_txt_dist = self._pdist(txt_embeds, txt_embeds)
        txt_txt_mask = torch.reshape(im_labels.t().repeat(1, txt_im_ratio), (num_txt, num_txt))
        pos_pair_dist = txt_txt_dist[txt_txt_mask].max(dim=0, keepdim=True)[0]
        neg_pair_dist = txt_txt_dist[~txt_txt_mask].view(num_txt, -1)
        sent_only_loss = F.relu(self.margin + pos_pair_dist - neg_pair_dist).topk(self.num_neg_sample)[0].mean()

        loss = im_loss * self.im_loss_factor + txt_loss + sent_only_loss * self.txt_only_loss_factor
        return loss

    def _pdist(self, x1, x2):
        x1_square = torch.sum(x1**2, dim=1).view(-1, 1)
        x2_square = torch.sum(x2**2, dim=1).view(1, -1)
        return torch.sqrt(x1_square - 2 * torch.mm(x1, x2.t()) + x2_square + 1e-4)

In [161]:
def generate_neg_pairs(image_embedding, text_embedding, sample_size):
    m, _ = image_embedding.shape
    txt_embeds = torch.zeros(sample_size * m, text_embedding.shape[1])
    im_labels = torch.zeros(sample_size * m, m, dtype=bool)
    all_indices = np.arange(m)
    for i in range(m):
        indices = np.delete(all_indices, i)
        random_indices = np.random.choice(indices, size=sample_size-1, replace=False)
        txt_embeds[i * sample_size, :] = text_embedding[i, :]
        txt_embeds[(i * sample_size + 1) : ((i + 1) * sample_size), :] = text_embedding[random_indices, :]
        im_labels[0, i] = True
    return image_embedding, txt_embeds, im_labels

In [162]:
def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    # transformers.set_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [163]:
SEED = 595
set_seed(SEED)

image_features = np.load('features/image_features_vgg16_remove.npy')
text_features = np.load('features/text_LaBSE_merge.npy')
batch_size = 256
img_feature_size = image_features.shape[1]
txt_feature_size = text_features.shape[1]
learning_rate = 0.0001
weight_decay = 0.1
epochs = 10
sample_size = 5
embed_dim = 1024

dataset = FeatureDataset(image_features, text_features)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
branch_net = BranchNetwork(img_feature_size, txt_feature_size, embed_dim = embed_dim)
# optimizer = torch.optim.Adam(branch_net.parameters(), lr=learning_rate)
optimizer = torch.optim.AdamW(branch_net.parameters(),
                                lr=learning_rate,
                                weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                    T_max=epochs,
                                                    eta_min=learning_rate/50)
embed_loss = EmbeddingLoss(num_neg_sample=sample_size)

# Training
for epoch in range(epochs):
    for image_batch, text_batch in dataloader:
        image_embedding, text_embedding = branch_net(image_batch, text_batch)
        im_embeds, txt_embeds, im_labels = generate_neg_pairs(image_embedding, text_embedding, sample_size)
        loss = embed_loss(im_embeds, txt_embeds, im_labels, sample_size)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    lr_scheduler.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 1/10, Loss: 0.4060887098312378
Epoch 2/10, Loss: 0.4047694206237793
Epoch 3/10, Loss: 0.42284196615219116
Epoch 4/10, Loss: 0.4120154082775116
Epoch 5/10, Loss: 0.3867340087890625
Epoch 6/10, Loss: 0.39003583788871765
Epoch 7/10, Loss: 0.39095544815063477
Epoch 8/10, Loss: 0.3868507146835327
Epoch 9/10, Loss: 0.3876052498817444
Epoch 10/10, Loss: 0.3833191692829132


In [164]:
i_result, t_result = branch_net.forward(torch.tensor(image_features), torch.tensor(text_features))
np.save('features/trained_img_features.npy',i_result.detach().numpy())
np.save('features/trained_txt_features.npy',t_result.detach().numpy())